<a href="https://colab.research.google.com/github/tvani2/Neural-Networks-Facial-Expression-Recognition-Challenge/blob/main/Expr_recognition_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle wandb onnx -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 118.3 MB/s eta 0:00:00


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/cs231n/assignments/assignment4/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge

 87% 248M/285M [00:00<00:00, 853MB/s] 
100% 285M/285M [00:00<00:00, 864MB/s]
Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [12]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
# Change the file path to the correct location after unzipping
df = pd.read_csv('./train.csv')
X = df['pixels']
y = df['emotion']

train_size = 0.70
val_size = 0.15
test_size = 0.15
X_temp, X_test_new, y_temp, y_test_new = train_test_split(
    X, y, test_size=test_size, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=(val_size / (train_size + val_size)), random_state=42, stratify=y_temp
)

In [14]:
from torch.utils.data import Dataset

def fast_process_pixels(pixel_series):
    pixel_lists = pixel_series.str.split()
    pixel_array = np.array(pixel_lists.tolist(), dtype=np.float32)
    return pixel_array.reshape(-1, 48, 48, 1) / 255.0

X_train_normalized = fast_process_pixels(X_train)
X_val_normalized = fast_process_pixels(X_val)
X_test_new_normalized = fast_process_pixels(X_test_new)

print("Data preprocessing completed!")
print(f"Train shape: {X_train_normalized.shape}")
print(f"Val shape: {X_val_normalized.shape}")
print(f"Test shape: {X_test_new_normalized.shape}")

# === 4. Dataset Class ===
class FastEmotionDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = torch.from_numpy(images).permute(0, 3, 1, 2).float()
        self.labels = torch.from_numpy(labels.values).long()
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

Data preprocessing completed!
Train shape: (20095, 48, 48, 1)
Val shape: (4307, 48, 48, 1)
Test shape: (4307, 48, 48, 1)


In [15]:
import torch
from torchvision import transforms
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import WeightedRandomSampler, DataLoader

# === 5. Transforms ===
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.Normalize([0.5], [0.5])
])

val_test_transforms = transforms.Compose([
    transforms.Normalize([0.5], [0.5])
])

# === 6. Create Datasets ===
train_dataset = FastEmotionDataset(X_train_normalized, y_train, transform=train_transforms)
val_dataset = FastEmotionDataset(X_val_normalized, y_val, transform=val_test_transforms)
test_dataset = FastEmotionDataset(X_test_new_normalized, y_test_new, transform=val_test_transforms)

# === 7. Compute Class Weights and Sampler ===
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))
sample_weights = np.array([class_weights_dict[label] for label in y_train])
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# === 8. DataLoaders ===
batch_size = 64
num_workers = 0
pin_memory = torch.cuda.is_available()

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler,
                          num_workers=num_workers, pin_memory=pin_memory,
                          persistent_workers=num_workers > 0)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=num_workers, pin_memory=pin_memory,
                        persistent_workers=num_workers > 0)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=pin_memory,
                         persistent_workers=num_workers > 0)

In [16]:
class EmotionDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        # Convert to torch tensors and rearrange dimensions (H, W, C) -> (C, H, W)
        self.images = torch.FloatTensor(images).permute(0, 3, 1, 2)
        self.labels = torch.LongTensor(labels.values)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define data augmentation transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])

# No augmentation for validation and test
val_test_transforms = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5])  # Same normalization as training
])

# Create datasets
train_dataset = EmotionDataset(X_train_normalized, y_train, transform=train_transforms)
val_dataset = EmotionDataset(X_val_normalized, y_val, transform=val_test_transforms)
test_dataset = EmotionDataset(X_test_new_normalized, y_test_new, transform=val_test_transforms)

In [17]:
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create sample weights for each training sample
sample_weights = [class_weights_dict[label] for label in y_train]
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

In [18]:
# Create DataLoaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_new_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print class distribution info
print("Class distribution in training set:")
print(y_train.value_counts().sort_index())
print(f"\nClass weights: {class_weights_dict}")

Class distribution in training set:
emotion
0    2797
1     306
2    2867
3    5051
4    3380
5    2219
6    3475
Name: count, dtype: int64

Class weights: {0: np.float64(1.0263547678635272), 1: np.float64(9.381419234360411), 2: np.float64(1.0012955304200508), 3: np.float64(0.568345730689821), 4: np.float64(0.849323753169907), 5: np.float64(1.2936972896414085), 6: np.float64(0.8261048304213772)}


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class EmotionResNet(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.resnet.fc.in_features, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0.001, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = float('inf')
        self.counter = 0
        self.best_weights = None

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            if self.restore_best_weights:
                self.best_weights = model.state_dict().copy()
        else:
            self.counter += 1

        if self.counter >= self.patience:
            if self.restore_best_weights and self.best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False

In [ ]:
wandb.init(project="Facial_Expression_Recognition_1", name="ResNet_1")

wandb.config.update({
    "model": "ResNet18",
    "epochs": 20,
    "batch_size": train_loader.batch_size,
    "learning_rate": 0.0003,
    "weight_decay": 1e-5,
    "scheduler": "ReduceLROnPlateau",
    "optimizer": "Adam",
    "early_stopping_patience": 7
})
raw_data_artifact = wandb.Artifact(
    name="facial-expression-dataset",
    type="dataset",
    description="Facial Expression Recognition Challenge Dataset"
)
# Log the artifact to the current run
wandb.log_artifact(raw_data_artifact)

print("Data loaded and logged as a wandb artifact.")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import wandb
import numpy as np

# Define model, optimizer, scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EmotionResNet(num_classes=7).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)
early_stopping = EarlyStopping(patience=7, min_delta=0.0005)

wandb.watch(model, log="all", log_freq=100)

train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
model_save_path = "best_resnet_model.pth"
best_val_loss = float('inf')

for epoch in range(20):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100. * correct / total
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_acc)

    # Validation
    model.eval()
    val_loss_total, correct_val, total_val = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    avg_val_loss = val_loss_total / len(val_loader.dataset)
    val_acc = 100. * correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1:02d}: Train Loss = {avg_train_loss:.4f}, Train Acc = {train_acc:.2f}%, "
          f"Val Loss = {avg_val_loss:.4f}, Val Acc = {val_acc:.2f}%")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "train_accuracy": train_acc,
        "val_loss": avg_val_loss,
        "val_accuracy": val_acc,
        "learning_rate": optimizer.param_groups[0]['lr']
    })

    scheduler.step(avg_val_loss)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), model_save_path)

    if early_stopping(avg_val_loss, model):
        print(f"Early stopping at epoch {epoch+1}")
        break

# Load best model for inference/testing
model.load_state_dict(torch.load(model_save_path))
wandb.finish()


In [ ]:
model.load_state_dict(torch.load(model_save_path))

# Final Evaluation
print("\nEvaluating on test set...")
model.eval()
all_preds, all_labels = [], []

# Ensure device is defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Ensure model is on the correct device for evaluation

from sklearn.metrics import accuracy_score # Import accuracy_score

with torch.no_grad():
    for images, labels in test_loader:
        # Move images and labels to the same device as the model
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_preds) * 100
print(f"\n✅ Test Accuracy: {test_accuracy:.2f}%")

In [20]:
import torch
import torch.nn as nn
import torchvision.models as models

class EmotionResNet(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)

        # Replace first conv layer for grayscale input (1 channel)
        self.resnet.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),  # Added batch norm
            nn.ReLU(inplace=True)
        )

        # Replace FC with dropout and final classification layer
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.resnet.fc.in_features, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionResNet(num_classes=7).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)
early_stopping = EarlyStopping(patience=7, min_delta=0.0005)

wandb.init(project="Facial_Expression_Recognition_1")
wandb.watch(model, log="all", log_freq=100)

best_val_loss = float('inf')
model_save_path = "best_resnet_model.pth"

for epoch in range(20):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100. * correct / total

    model.eval()
    val_loss_total, correct_val, total_val = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    avg_val_loss = val_loss_total / len(val_loader.dataset)
    val_acc = 100. * correct_val / total_val

    print(f"Epoch {epoch+1:02d}: Train Loss = {avg_train_loss:.4f}, Train Acc = {train_acc:.2f}%, "
          f"Val Loss = {avg_val_loss:.4f}, Val Acc = {val_acc:.2f}%")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "train_accuracy": train_acc,
        "val_loss": avg_val_loss,
        "val_accuracy": val_acc,
        "learning_rate": optimizer.param_groups[0]['lr']
    })

    scheduler.step(avg_val_loss)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), model_save_path)

    if early_stopping(avg_val_loss, model):
        print(f"Early stopping at epoch {epoch+1}")
        break

# Load best model
model.load_state_dict(torch.load(model_save_path))
wandb.finish()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 01: Train Loss = 1.9746, Train Acc = 25.24%, Val Loss = 1.6634, Val Acc = 35.87%
Epoch 02: Train Loss = 1.5445, Train Acc = 41.25%, Val Loss = 1.6258, Val Acc = 41.54%
Epoch 03: Train Loss = 1.3538, Train Acc = 49.29%, Val Loss = 1.3776, Val Acc = 48.08%
Epoch 04: Train Loss = 1.2512, Train Acc = 53.25%, Val Loss = 1.3585, Val Acc = 48.66%
Epoch 05: Train Loss = 1.1756, Train Acc = 55.81%, Val Loss = 1.2298, Val Acc = 53.05%
Epoch 06: Train Loss = 1.1076, Train Acc = 58.23%, Val Loss = 1.2087, Val Acc = 54.10%
Epoch 07: Train Loss = 1.0698, Train Acc = 59.67%, Val Loss = 1.2758, Val Acc = 53.84%
Epoch 08: Train Loss = 1.0136, Train Acc = 62.21%, Val Loss = 1.1937, Val Acc = 55.44%
Epoch 09: Train Loss = 0.9756, Train Acc = 63.60%, Val Loss = 1.2234, Val Acc = 55.28%
Epoch 10: Train Loss = 0.9414, Train Acc = 65.17%, Val Loss = 1.2016, Val Acc = 56.14%
Epoch 11: Train Loss = 0.9017, Train Acc = 66.76%, Val Loss = 1.1801, Val Acc = 56.70%
Epoch 12: Train Loss = 0.8631, Train Acc = 

epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
learning_rate,███████████████▁▁▁
train_accuracy,▁▃▄▅▅▅▅▆▆▆▆▇▇▇▇▇██
train_loss,█▆▅▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
val_accuracy,▁▃▅▅▆▆▆▇▇▇▇▇▇▇▇███
val_loss,█▇▄▄▂▁▂▁▂▁▁▁▂▂▂▂▃▂
epoch,18
learning_rate,0.00015
train_accuracy,79.19881
train_loss,0.5775
val_accuracy,59.90248


In [22]:

import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class ImprovedEmotionResNet(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.5):
        super(ImprovedEmotionResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)

        # Replace first conv layer for grayscale input (1 channel)
        self.resnet.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.add_dropout_to_layers()

        # Enhanced classifier with multiple dropout layers
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_features, num_features // 2),
            nn.BatchNorm1d(num_features // 2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate * 0.7),  # Slightly less dropout in second layer
            nn.Linear(num_features // 2, num_classes)
        )

    def add_dropout_to_layers(self):
        """Add dropout after each residual block"""
        # Add dropout after layer1
        original_layer1 = self.resnet.layer1
        self.resnet.layer1 = nn.Sequential(
            original_layer1,
            nn.Dropout2d(0.1)  # Light spatial dropout
        )

        # Add dropout after layer2
        original_layer2 = self.resnet.layer2
        self.resnet.layer2 = nn.Sequential(
            original_layer2,
            nn.Dropout2d(0.15)
        )

        # Add dropout after layer3
        original_layer3 = self.resnet.layer3
        self.resnet.layer3 = nn.Sequential(
            original_layer3,
            nn.Dropout2d(0.2)
        )

        # Add dropout after layer4
        original_layer4 = self.resnet.layer4
        self.resnet.layer4 = nn.Sequential(
            original_layer4,
            nn.Dropout2d(0.25)
        )

    def forward(self, x):
        return self.resnet(x)

In [23]:
wandb.init(
    project="Facial_Expression_Recognition_1",
    config={
        "learning_rate": 1e-4,
        "architecture": "ImprovedEmotionResNet",
        "dataset": "FER2013",
        "epochs": 30,
        "batch_size": 32,
        "optimizer": "AdamW",
        "weight_decay": 1e-3,
        "label_smoothing": 0.1,
        "dropout_rate": 0.5
    }
)
wandb.watch(model, log="all", log_freq=100)

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import wandb
import numpy as np

# Define model, optimizer, scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ImprovedEmotionResNet(num_classes=7, dropout_rate=0.5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)
early_stopping = EarlyStopping(patience=10, min_delta=0.001)

wandb.watch(model, log="all", log_freq=100)

train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
model_save_path = "best_resnet_model.pth"
best_val_loss = float('inf')

for epoch in range(20):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    avg_train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100. * correct / total
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_acc)

    # Validation
    model.eval()
    val_loss_total, correct_val, total_val = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    avg_val_loss = val_loss_total / len(val_loader.dataset)
    val_acc = 100. * correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1:02d}: Train Loss = {avg_train_loss:.4f}, Train Acc = {train_acc:.2f}%, "
          f"Val Loss = {avg_val_loss:.4f}, Val Acc = {val_acc:.2f}%")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "train_accuracy": train_acc,
        "val_loss": avg_val_loss,
        "val_accuracy": val_acc,
        "learning_rate": optimizer.param_groups[0]['lr']
    })

    scheduler.step(avg_val_loss)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), model_save_path)

    if early_stopping(avg_val_loss, model):
        print(f"Early stopping at epoch {epoch+1}")
        break

# Load best model for inference/testing
model.load_state_dict(torch.load(model_save_path))

# Final Evaluation
print("\nEvaluating on test set...")
model.eval()
all_preds, all_labels = [], []

# Ensure device is defined
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Ensure model is on the correct device for evaluation

from sklearn.metrics import accuracy_score

with torch.no_grad():
    for images, labels in test_loader:
        # Move images and labels to the same device as the model
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_preds) * 100
print(f"\n✅ Test Accuracy: {test_accuracy:.2f}%")
wandb.finish()

Epoch 01: Train Loss = 1.9678, Train Acc = 17.75%, Val Loss = 1.8298, Val Acc = 27.44%
Epoch 02: Train Loss = 1.7794, Train Acc = 29.30%, Val Loss = 1.5854, Val Acc = 39.03%
Epoch 03: Train Loss = 1.6065, Train Acc = 38.86%, Val Loss = 1.5380, Val Acc = 40.79%
Epoch 04: Train Loss = 1.5062, Train Acc = 43.09%, Val Loss = 1.4401, Val Acc = 45.07%
Epoch 05: Train Loss = 1.4321, Train Acc = 46.38%, Val Loss = 1.4056, Val Acc = 46.58%
Epoch 06: Train Loss = 1.3856, Train Acc = 48.28%, Val Loss = 1.3322, Val Acc = 49.71%
Epoch 07: Train Loss = 1.3278, Train Acc = 50.92%, Val Loss = 1.3272, Val Acc = 50.20%
Epoch 08: Train Loss = 1.2757, Train Acc = 53.01%, Val Loss = 1.2702, Val Acc = 51.96%
Epoch 09: Train Loss = 1.2641, Train Acc = 53.15%, Val Loss = 1.2988, Val Acc = 51.52%
Epoch 10: Train Loss = 1.2310, Train Acc = 54.38%, Val Loss = 1.3092, Val Acc = 51.71%
Epoch 11: Train Loss = 1.2104, Train Acc = 55.32%, Val Loss = 1.2772, Val Acc = 52.82%
Epoch 12: Train Loss = 1.1839, Train Acc = 

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
learning_rate,███████████████████▁
train_accuracy,▁▃▄▅▅▆▆▇▇▇▇▇▇▇▇▇████
train_loss,█▇▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▂▁
val_accuracy,▁▄▄▅▆▆▆▇▇▇▇▇▇████▇▇█
val_loss,█▅▅▄▄▃▃▂▃▃▂▂▂▁▁▁▁▂▂▁
epoch,20
learning_rate,0.0001
train_accuracy,62.52799
train_loss,1.02911
val_accuracy,57.13954
